In [1]:
import requests
import lxml.html

In [2]:
from requests.models import Response

def scrape_list_page(response: Response):
    """一覧ページのResponseから詳細ページのURLを抜き出す"""
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect("#listBook a[itemprop='url']"):
        url = a.get("href")
        yield url

In [3]:
def scrape_detail_page(response):
    """詳細ページの情報をdictで取得する"""
    root = lxml.html.fromstring(response.content)
    ebook = {
        "url": response.url,
        "title": root.cssselect("#bookTitle")[0].text_content(),
        "price": root.cssselect(".buy")[0].text, # `.text` で直接の子である文字列のみ取得
        "content": [h3.text_content() for h3 in root.cssselect("#content > h3")],
    }
    return ebook

In [4]:
session = requests.Session()
response = session.get("http://sample.scraping-book.com/dp")
urls = scrape_list_page(response)

In [5]:
for url in urls:
    response = session.get(url)
    ebook = scrape_detail_page(response)
    print(ebook)
    break

{'url': 'http://sample.scraping-book.com/dp/ebook/2016/978-4-7741-8336-7', 'title': 'これからはじめるプログラミング 作って覚える基礎の基礎', 'price': '2,280円 ', 'content': ['第0章\u3000まずは道具を準備しよう', '第1章\u3000プログラミングを始めよう', '第2章\u3000プログラムの流れを理解しよう', '第3章\u3000値を入れる箱をマスターしよう', '第4章\u3000プログラムの流れをコントロールしよう', '第5章\u3000ループを使いこなそう', '第6章\u3000アニメーションにチャレンジ', '第7章\u3000一歩進んだプログラミング', '第8章\u3000お掃除ロボットを作ろう！', '第9章\u3000次のステップへ', '付録\u3000ブロック一覧']}
